In [1]:
import tensorflow as tf
print(tf.__version__)

2.0.0-rc0


## 评估函数使用原理

In [2]:
m = tf.keras.metrics.Accuracy()
m.update_state([1,2,3,4],[0,2,3,4])

print('Final result:' , m.result().numpy())

#Fianl result : 0.75
m.update_state([1,2,3,4],[0,2,3,1])
print('Final result:', m.result().numpy())

Final result: 0.75
Final result: 0.625


In [3]:
m = tf.keras.metrics.Accuracy()
m([1,2,3,4],[0,2,3,4])

print('Final result:', m.result().numpy())
# Final result:0.75
m([1,2,3,4],[0,2,3,1])
print('Final result:', m.result().numpy())


Final result: 0.75
Final result: 0.625


In [4]:
m.reset_states()

In [5]:
m.update_state([1,2,3,4],[0,2,3,4])
print('Final result:', m.result().numpy())

Final result: 0.75


## 自定义评估函数

In [9]:
class SparseCategoricalAccuracy_(tf.keras.metrics.Metric):
    def __init__(self,name='SparseCategoricalAccuracy',**kwargs):
        super(SparseCategoricalAccuracy_,self).__init__(name=name,**kwargs)
        self.total = self.add_weight(name='total',dtype=tf.int32, initializer=tf.zeros_initializer())
        self.count = self.add_weight(name='count',dtype=tf.int32, initializer=tf.zeros_initializer())
        
    def update_state(self,y_true,y_pred,sample_weight=None):
        values = tf.cast(tf.equal(y_true,tf.argmax(y_pred,axis=-1,output_type=tf.int32)), tf.int32)
        self.total.assign_add(tf.shape(y_true)[0])
        self.count.assign_add(tf.reduce_sum(values))
        
    def result(self):
        return self.count /self.total
    
    def reset_states(self):
        # The state of metric will be reset at the start of each epoch.
        self.total.assign(0)
        self.count.assign(0)

In [11]:
s = SparseCategoricalAccuracy_()
s.update_state(tf.constant([2,1]),tf.constant([[0.1,0.9,0.8],[0.05,0.95,0]]))
print('Final result:', s.result().numpy())

Final result: 0.5


In [12]:
m = tf.keras.metrics.SparseCategoricalAccuracy()
m.update_state(tf.constant([2,1]),tf.constant([[0.1,0.9,0.8],[0.05,0.95,0]]))
print('Final result:', m.result().numpy())

Final result: 0.5
